---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    graphs=[]
    results=[]
    for el in P1_Graphs:
        #print (nx.average_shortest_path_length(el))
        #print(nx.info(el))
        #print(nx.average_clustering(el))
        graphs.append(str(el))
        #print (nx.clustering(el))
        #print(el.nodes())
    #print(graphs)
    for model in graphs:
        #print (model[-3:-1])
        if 'barabasi' in model:
            results.append('PA')
        if 'watts' in model and model[-3:-1] == '05':
            results.append('SW_L')
        if 'watts' in model and model[-3:-1] == '75':
            results.append('SW_L')
        if 'watts' in model and model[-3:-1] == ',1':
            results.append('SW_H')
            
    return(results)
    

In [4]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [7]:
G = nx.read_gpickle('email_prediction.txt')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [8]:
def salary_predictions():
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import roc_curve, roc_auc_score, auc

    
    G.nodes(data=True)
    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['M_salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Deg_Centrality'] = pd.Series(nx.degree_centrality(G))
    df['Close_Centrality']= pd.Series(nx.closeness_centrality(G))
    df['Close_Cent_Normalized']= pd.Series(nx.closeness_centrality(G, normalized=True))
    df['Degree']=pd.Series(G.degree())
    df = df[['Department','Deg_Centrality','Close_Centrality','Close_Cent_Normalized','Degree',  'M_salary']]
    
    train=df.loc[(df['M_salary'] == 1.0) | (df['M_salary'] == 0.0)]
    test=df[df['M_salary'] != 1]
    test=test[test['M_salary'] !=0]
    

    X_train = train.iloc[:,:-1]
    y_train= train.iloc[:,-1]

    X_test=test.iloc[:,:-1]
    y_test=test.iloc[:,-1]

#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

    model = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 4, random_state = 0)
#model = SVC(kernel='rbf', gamma=0.001)
    model.fit(X_train, y_train)
    predictions = model.fit(X_train, y_train).predict_proba(X_test)
    #print(predictions)
    res=[]
    for el in predictions:
        res.append(el[1])
    
    df_res=pd.Series(res, index=y_test.index)

    return (df_res)

In [9]:
salary_predictions()

1       0.011098
2       0.964228
5       0.989569
8       0.183545
14      0.199513
18      0.283151
27      0.205123
30      0.568687
31      0.423664
34      0.008339
37      0.007253
40      0.164483
45      0.039789
54      0.502937
55      0.778852
60      0.139959
62      0.982836
65      0.139039
77      0.007146
79      0.047180
97      0.002738
101     0.002663
103     0.067956
108     0.194529
113     0.719901
122     0.003354
141     0.100847
142     0.982836
144     0.021370
145     0.105085
          ...   
913     0.016353
914     0.005092
915     0.011593
918     0.003354
923     0.578834
926     0.011106
931     0.011106
934     0.005137
939     0.013807
944     0.007229
945     0.011805
947     0.013332
950     0.114171
951     0.008374
953     0.030880
959     0.007166
962     0.011065
963     0.043785
968     0.008374
969     0.007953
974     0.007444
984     0.004454
987     0.008514
989     0.011805
991     0.006155
992     0.007166
994     0.008389
996     0.0071

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.info()

Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [10]:
def new_connections_predictions():
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.metrics import roc_curve, roc_auc_score, auc
    
    #G.edges(data=True)
    #df = pd.DataFrame(index=G.edges())
    #new_df=pd.merge(df,future_connections,how='outer', right_index=True, left_index=True)
    #new_df['Common_neighbors'] = new_df.index.map(lambda edge: len(list(nx.common_neighbors(G, edge[0], edge[1]))))
   # new_df['Jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, new_df.index)]
    #new_df= new_df[['Jaccard', 'Common_neighbors', 'Future Connection']]
    future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
    future_connections['Common_neighbors'] = future_connections.index.map(lambda edge: len(list(nx.common_neighbors(G, edge[0], edge[1]))))
    future_connections['Jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
    future_connections= future_connections[['Jaccard', 'Common_neighbors', 'Future Connection']]
    
    train=future_connections.loc[(future_connections['Future Connection'] == 1.0) | (future_connections['Future Connection'] == 0.0)]
    test=future_connections[future_connections['Future Connection']!= 1]
    test=test[test['Future Connection']!= 0]
    #print(test.shape)
    #print(train.shape)
    
    #train=new_df.loc[(new_df['Future Connection'] == 1.0) | (new_df['Future Connection'] == 0.0)]
    #test=new_df[new_df['Future Connection']!= 1]
    #test=test[test['Future Connection']!= 0]
    #print(train.head())
    #print(test.head())
    
    X_train = train.iloc[:,:-1]
    y_train= train.iloc[:,-1]

    X_test=test.iloc[:,:-1]
    y_test=test.iloc[:,-1]

#X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)

    model = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 4, random_state = 0)
#model = SVC(kernel='rbf', gamma=0.001)
    model.fit(X_train, y_train)
    predictions = model.fit(X_train, y_train).predict_proba(X_test)
    #print(predictions)
    res=[]
    for el in predictions:
        res.append(el[1])
    
    df_res=pd.Series(res, index=y_test.index)

    return (df_res)

In [11]:
new_connections_predictions()

(107, 348)    0.022668
(542, 751)    0.012887
(20, 426)     0.584040
(50, 989)     0.012887
(942, 986)    0.012887
(324, 857)    0.012887
(13, 710)     0.104831
(19, 271)     0.157706
(319, 878)    0.012887
(659, 707)    0.012887
(49, 843)     0.012887
(208, 893)    0.012887
(377, 469)    0.012887
(405, 999)    0.016715
(129, 740)    0.020730
(292, 618)    0.018033
(239, 689)    0.012887
(359, 373)    0.012887
(53, 523)     0.031550
(276, 984)    0.012887
(202, 997)    0.012887
(604, 619)    0.135318
(270, 911)    0.012887
(261, 481)    0.078459
(200, 450)    0.851007
(213, 634)    0.012887
(644, 735)    0.123293
(346, 553)    0.012887
(521, 738)    0.012887
(422, 953)    0.018033
                ...   
(672, 848)    0.012887
(28, 127)     0.974204
(202, 661)    0.012887
(54, 195)     0.999044
(295, 864)    0.012887
(814, 936)    0.012887
(839, 874)    0.012887
(139, 843)    0.012887
(461, 544)    0.012887
(68, 487)     0.012887
(622, 932)    0.012887
(504, 936)    0.016496
(479, 528) 